# Transient Storage

These opcodes behave almost identically to storage but changes are discarded after every transaction.

We will declare separate class for transient storage. The reason is that we need to clear the storage after the transaction is done by evm.

In [ ]:
class KeyValue:
    def __init__(self):
        self.storage={}

    def load(self,key):
        return self.storage[key]
    
    def store(self,key,value):
        self.storage[key]=value


class TStorage(KeyValue):
    def __init__(self):
        super().__init__()
        self.cache = []  # warm set 

    def load(self, key):
        warm = key in self.cache
        if not warm: self.cache.append(key)
        if key not in self.storage:return warm, 0x00
        return warm, super().load(key)

    def store(self, key, value):
        warm = key in self.cache
        if not warm:
            self.cache.append(key)
        self.storage[key] = value
        return warm

    def reset(self):
        # wipe I have done to clear at the end of transaction
        self.storage.clear()
        self.cache.clear()



#### Transient Storage Load

In [4]:
def tload(evm):
    key = evm.stack.pop()
    warm, value = evm.tstorage.load(key)  
    evm.stack.push(value)

  
    evm.gas_dec(100 if warm else 2100)

    evm.pc += 1

#### Transient Storage Store

In [3]:
def tstore(evm): 
    key, value = evm.stack.pop(), evm.stack.pop()
    warm = evm.tstorage.store(key, value)
    evm.gas_dec(100 if warm else 2100)

    evm.pc += 1